In [1]:
#Import packages
from arcgis import GIS
from arcgis.features import FeatureLayer, GeoAccessor

/opt/conda/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [3]:
gis = GIS('https://NSOE.maps.arcgis.com','jpfay_790')

Enter password: ········


In [4]:
print("Logged in as: " + gis.properties.user.username)

Logged in as: jpfay_790


In [5]:
flood_layer_service = gis.content.get('11955f1b47ec41a3af86650824e0c634')
type(flood_layer_service)

arcgis.gis.Item

In [7]:
flood_layer = flood_layer_service.layers[0]
type(flood_layer)

arcgis.raster._layer.ImageryLayer

In [6]:
theExits = GeoAccessor.from_featureclass('../Data/MJBA/Exits.shp')
type(theExits.spatial)

arcgis.features.geo._accessor.GeoAccessor

In [11]:
theAttributes = flood_layer.attribute_table()['features']

[{'attributes': {'OBJECTID': 1,
   'FREQUENCY': 963562,
   'Value': 1,
   'Red': 142,
   'Green': 52,
   'Blue': 148,
   'ClassName': '1% Annual Chance Flood Hazard'}},
 {'attributes': {'OBJECTID': 2,
   'FREQUENCY': 38339,
   'Value': 2,
   'Red': 118,
   'Green': 156,
   'Blue': 205,
   'ClassName': 'Regulatory Floodway'}},
 {'attributes': {'OBJECTID': 3,
   'FREQUENCY': 67,
   'Value': 3,
   'Red': 129,
   'Green': 192,
   'Blue': 201,
   'ClassName': 'Special Floodway'}},
 {'attributes': {'OBJECTID': 4,
   'FREQUENCY': 3274,
   'Value': 4,
   'Red': 210,
   'Green': 212,
   'Blue': 207,
   'ClassName': 'Area of Undetermined Flood Hazard'}},
 {'attributes': {'OBJECTID': 5,
   'FREQUENCY': 1053911,
   'Value': 5,
   'Red': 157,
   'Green': 125,
   'Blue': 192,
   'ClassName': '0.2% Annual Chance Flood Hazard'}},
 {'attributes': {'OBJECTID': 6,
   'FREQUENCY': 31608,
   'Value': 6,
   'Red': 245,
   'Green': 200,
   'Blue': 179,
   'ClassName': 'Future Conditions 1% Annual Chance Floo

In [12]:
classNames = []
for theAttribute in theAttributes:
    className = theAttribute['attributes']['ClassName']
    classNames.append(className)
    
classNames

['1% Annual Chance Flood Hazard',
 'Regulatory Floodway',
 'Special Floodway',
 'Area of Undetermined Flood Hazard',
 '0.2% Annual Chance Flood Hazard',
 'Future Conditions 1% Annual Chance Flood Hazard',
 'Area with Reduced Risk Due to Levee',
 'Area of Minimal Flood Hazard']

In [13]:
classNames[int(theValue)-1]

'Special Floodway'

In [14]:
def getFloodClass(thePoint):
    return classNames[int(flood_layer.identify(thePoint)['value'])-1]

In [16]:
theExits['Flood_cat'] = theExits['SHAPE'].apply(lambda x: getFloodClass(x))

In [76]:
theExits.head()

,index,FID,Exit_ID,Longitude,Latitude,County,State,Highway,Type,AADT,Pop_dens,Comm_1mi,Ports_5mi,Ports_10mi,Ports_20mi,Close_nonT,SHAPE,Flood_cat
0,0,7059,US-64_Cherokee_NC_Exit_1,-84.314304,35.019492,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3700,60.34979,1,0,0,0,34.094152,"{'x': -84.31430368976345, 'y': 35.019491849669...",Area of Minimal Flood Hazard
1,1,7060,US-64_Cherokee_NC_Exit_2,-84.311492,35.014933,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3700,60.34979,1,0,0,0,34.365230,"{'x': -84.31149165991212, 'y': 35.014933379571...",Area of Minimal Flood Hazard
2,2,7061,US-64_Cherokee_NC_Exit_3,-84.295528,35.013561,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,35.233979,"{'x': -84.29552778977563, 'y': 35.013560589649...",Area of Minimal Flood Hazard
3,3,7062,US-64_Cherokee_NC_Exit_4,-84.280291,35.012880,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,36.048303,"{'x': -84.28029090010494, 'y': 35.012879950349...",Area of Minimal Flood Hazard
4,4,7063,US-64_Cherokee_NC_Exit_5,-84.275190,35.017816,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,36.186597,"{'x': -84.27519011992803, 'y': 35.017816120417...",Area of Minimal Flood Hazard


In [77]:
theExits.to_csv('../Data/processed/exits_with_flood.csv')

In [78]:
#Save out modified dataframe to a shapefile
theExits.spatial.to_featureclass('../Data/processed/exits_flood.shp')

'../Data/processed/exits_flood.shp'

In [81]:
theExits['Flood_cat'].value_counts()

Area of Minimal Flood Hazard                       1090
1% Annual Chance Flood Hazard                        58
0.2% Annual Chance Flood Hazard                      24
Regulatory Floodway                                   4
Future Conditions 1% Annual Chance Flood Hazard       2
Name: Flood_cat, dtype: int64